In [24]:
import pandas as pd
import os

In [25]:
file_df = []
for file in os.listdir("weibo2"):
    if file.endswith(".xlsx"):
        file_df.append(pd.read_excel(f"weibo2/{file}"))

df = pd.concat(file_df)
print(len(df))

df = df.drop_duplicates(subset="博文独立网址")
df = df[df["博文"].str.contains("医|药")]
print(len(df))

df[["转发数", "评论数", "点赞数"]] = df[["转发数", "评论数", "点赞数"]].apply(
    lambda col: pd.to_numeric(col, errors="coerce").fillna(0).astype(int)
)

df = df.sort_values("转发数", ascending=False)
df[df["博文"].str.contains("展开")].to_excel("weibo_not_collapsed.xlsx", index=False)
df[~df["博文"].str.contains("展开")].to_excel("weibo_all_collapsed.xlsx", index=False)

df["博文独立网址"] = df["博文独立网址"].str.strip("?refer_flag=1001030103_")

df["发布时间"] = pd.to_datetime(
    "2025年" + df["发布时间"], format="%Y年%m月%d日 %H:%M"  # Add the assumed year
)

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook con

73078
19555


In [26]:
# Load the fulltext dataset
fulltext = pd.read_excel("weibo_fulltext/temp_微博博文展开全部采集_规则_1_列表_20250301073520921.xlsx")
# If fulltext is a DataFrame with potentially multiple rows
for idx, row in fulltext.iterrows():
    if pd.notna(row['博文']):
        df.loc[df['博文独立网址'] == row['博文独立网址'], '博文'] = row['博文']
        
# Load the fulltext dataset
fulltext = pd.read_excel("weibo_fulltext/temp_微博博文展开全部采集_规则_1_列表_20250302043232700.xlsx")
# If fulltext is a DataFrame with potentially multiple rows
for idx, row in fulltext.iterrows():
    if pd.notna(row['博文']):
        df.loc[df['博文独立网址'] == row['博文独立网址'], '博文'] = row['博文']

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
# 给所有 URL 添加相同的参数
df['博文独立网址'] = df['博文独立网址'] + "?refer_flag=1001030103_"
df = df[df['博文'].str.contains("医|药")]
df = df[~df['博文'].str.contains("展开c")]

In [28]:
data = df[['博主id', '博主', '博主是否认证', '博文', '博文独立网址', '发布时间', '发布终端', '转发数', '评论数', '点赞数']]
data.columns = ['author_id', 'author', 'author_type', 'post', 'post_url', 'date', 'source', 'repost', 'comments', 'likes']

In [29]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import hanlp
import os
from datetime import datetime

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [30]:
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
tok.dict_force = {'原研药', '中成药'}

# 定义预处理函数
def preprocess_text(text, stop_words):
    if pd.isna(text):
        return None
    if len(text) == 0:
        return None
    # 使用HanLP分词
    tokens = tok(text)
    # 移除停用词和长度小于等于1的词
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return tokens

In [31]:
# 加载停用词 - 如果没有停用词文件，可以使用空集合
try:
    with open("stop_words.txt", 'r', encoding='utf-8') as f:
        stop_words = set([line.strip() for line in f])
    print(f"加载了 {len(stop_words)} 个停用词")
except FileNotFoundError:
    stop_words = set()
    print("未找到停用词文件，将使用空停用词集")

加载了 3065 个停用词


In [32]:
def extract_chinese(text):
    if not isinstance(text, str):
        return []
    
    pattern = re.compile(r'[\u4e00-\u9fff]+')
    chinese_only = ''.join(pattern.findall(text))
    
    if not chinese_only:
        return []
        
    return preprocess_text(chinese_only, stop_words)

# 提取中文词语并创建新列
data['tokens'] = data['post'].apply(extract_chinese)

/var/folders/st/gm4vk9215ls0bdt0m5nk_pq40000gn/T/ipykernel_49992/1212120334.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tokens'] = data['post'].apply(extract_chinese)


In [33]:
# 按日期分组
daily_data = data.groupby(data['date'].dt.date)

# 计算每日词频
daily_word_freq = {}
for day, group in daily_data:
    # 合并当天所有帖子的词语
    all_words = []
    for word_list in group['tokens']:
        if word_list is not None:
            all_words.extend(word_list)
        
    # 计算词频
    word_counts = Counter(all_words)
    
    # 存储当天的词频
    daily_word_freq[day] = word_counts

# 显示每日词频统计结果示例
sample_days = list(daily_word_freq.keys())
for day in sample_days:
    print(f"日期: {day}")
    print(f"总词数: {sum(daily_word_freq[day].values())}")
    print(f"不同词数: {len(daily_word_freq[day])}")
    print(f"前10个高频词: {daily_word_freq[day].most_common(100)}")
    print("-" * 80)

日期: 2025-01-01
总词数: 4829
不同词数: 1722
前10个高频词: [('药品', 95), ('降价', 89), ('集采', 83), ('医疗', 64), ('产品', 59), ('医药', 59), ('大幅', 56), ('行业', 53), ('公司', 50), ('医保', 49), ('中选', 46), ('医院', 44), ('创新药', 41), ('价格', 40), ('中成药', 38), ('增长', 34), ('网页', 32), ('链接', 32), ('阿司匹林', 31), ('原研药', 30), ('生产', 28), ('成本', 27), ('国内', 27), ('医生', 26), ('临床', 24), ('支付', 24), ('降质', 24), ('集采药', 22), ('机构', 22), ('目录', 20), ('采药', 17), ('健康', 16), ('品种', 16), ('生物', 15), ('基金', 15), ('国家医保局', 14), ('上市', 14), ('费用', 14), ('降幅', 14), ('收入', 14), ('便宜', 13), ('肠溶片', 13), ('引发', 13), ('关注', 13), ('数据', 13), ('治疗', 13), ('药物', 13), ('中药', 13), ('用药', 13), ('稳定', 12), ('中标', 12), ('带来', 12), ('降低', 12), ('仿制', 12), ('进口', 12), ('采购联盟', 12), ('谈判', 11), ('商业', 11), ('项目', 11), ('新华社', 11), ('地区', 11), ('疗效', 11), ('有望', 11), ('注射剂', 10), ('热议', 10), ('落地', 10), ('公告', 10), ('耗材', 10), ('海外', 10), ('过敏', 10), ('看病', 10), ('患者', 10), ('选择', 10), ('纳入', 10), ('处方', 10), ('建议', 10), ('子公司', 10), ('成功', 9), ('注射

In [43]:
from collections import Counter
import pandas as pd

# 定义时间段
bins = [1, 13, 18, 32]  # 分别对应 1-12, 13-17, 18-31 日
labels = ["1-12 Jan", "13-17 Jan", "18-31 Jan"]

data['day'] = data['date'].dt.day  # 提取日期中的“日”
data['period'] = pd.cut(data['day'], bins=bins, labels=labels, right=False)  # 归类到时间段

# 按时间段分组
periodic_data = data.groupby('period')

# 计算每个时间段的词频
periodic_word_freq = {}
for period, group in periodic_data:
    all_words = []
    for word_list in group['tokens']:
        if word_list is not None:
            all_words.extend(word_list)
    
    word_counts = Counter(all_words)
    periodic_word_freq[period] = word_counts

# 创建词频表格
freq_data = []
for period in labels:
    if period in periodic_word_freq:
        for word, count in periodic_word_freq[period].most_common(100):  # 仅取前100高频词
            freq_data.append([period, word, count])

freq_df = pd.DataFrame(freq_data, columns=["Time Period", "Word", "Frequency"])

/var/folders/st/gm4vk9215ls0bdt0m5nk_pq40000gn/T/ipykernel_49992/1419392067.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = data['date'].dt.day  # 提取日期中的“日”
/var/folders/st/gm4vk9215ls0bdt0m5nk_pq40000gn/T/ipykernel_49992/1419392067.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['period'] = pd.cut(data['day'], bins=bins, labels=labels, right=False)  # 归类到时间段
/var/folders/st/gm4vk9215ls0bdt0m5nk_pq40000gn/T/ipykernel_49992/1419392067.py:12: FutureWarning: The default of observed=F

In [45]:
freq_df.to_excel("word_freq.xlsx", index = False)